<a href="https://colab.research.google.com/github/Vasudeo-Rane/python_projects/blob/main/pyspark2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
conf = SparkConf().setAppName("test2").setMaster("local[*]")

In [4]:
sc = SparkContext(conf=conf)

In [5]:
import random
randomList = random.sample(range(0,40),10)
rdd = sc.parallelize(randomList,4)

In [6]:
print(rdd.glom().collect())

[[28, 1], [23, 0], [14, 32], [33, 34, 27, 11]]


In [14]:
#union
rdd2 = sc.parallelize([1,14,28,10,13,4,7],2)
rdd_union = rdd.union(rdd2)
print(rdd_union.getNumPartitions())
print(rdd_union.collect())

6
[28, 1, 23, 0, 14, 32, 33, 34, 27, 11, 1, 14, 28, 10, 13, 4, 7]


In [16]:
#intersection
rdd_intersection = rdd.intersection(rdd2)
print(rdd_intersection.collect())
print(rdd_intersection.getNumPartitions())

[1, 14, 28]
6


In [22]:
#find Empty partition
counter = 0
for item in rdd_intersection.glom().collect():
    if len(item) == 0:
      counter += 1
counter

3

In [23]:
#coalesce(numPartitions)
rdd_intersection.coalesce(1)

CoalescedRDD[54] at coalesce at NativeMethodAccessorImpl.java:0

In [24]:
#takeSample(withReplacement, num , [seed])  - Becareful
rdd.takeSample(False,5)

[23, 0, 1, 32, 27]

In [26]:
#takeOrdered(n,[ordering])
rdd.takeOrdered(6)

[0, 1, 11, 14, 23, 27]

In [29]:
#reduce
rdd.reduce(lambda x,y : x-y)

61

In [33]:
#reduceByKey()
rdd_Rbk = sc.parallelize([(1,4),(7,10),(5,7),(1,12),(7,12),(7,1),(9,1),(7,4)],2)
rdd_Rbk.reduceByKey(lambda x,y : x +y).collect()
#userFriendly representation
import pandas as pd
Counter = pd.DataFrame({'Key':rdd_Rbk.keys().collect(), 'Values':rdd_Rbk.values().collect()})
Counter

,Key,Values
0,1,4
1,7,10
2,5,7
3,1,12
4,7,12
5,7,1
6,9,1
7,7,4


In [34]:
#sortByKey
rdd_Rbk.reduceByKey(lambda x,y : x +y).sortByKey(True).collect()

[(1, 16), (5, 7), (7, 27), (9, 1)]

In [35]:
from re import L
#countByKey
rdd_Rbk.countByKey()

defaultdict(int, {1: 2, 7: 4, 5: 1, 9: 1})

In [40]:
#groupByKey():
rdd_group = rdd_Rbk.groupByKey()
rdd_group.getNumPartitions()
rdd_group.collect()
for item in rdd_group.collect():
  print(item[0], [value for value in item[1]])


1 [4, 12]
7 [10, 12, 1, 4]
5 [7]
9 [1]


In [41]:
#lookup(Key)
rdd_Rbk.lookup(7)

[10, 12, 1, 4]

In [42]:
#cache :
#By default , each transformation RDD may be recomputed each time you run a action on it
# However, you may also persist an RDD in memory using the persist(or cache) method,
# in which case Spark will keep the elements around on the cluster for much faster access the next time  you query it
#rdd_Rbk.cache()
#rdd_Rbk.persist()

In [43]:
# Persistence (https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence)
from pyspark import StorageLevel
rdd_Rbk.persist(StorageLevel.MEMORY_AND_DISK)

ParallelCollectionRDD[83] at readRDDFromFile at PythonRDD.scala:289